# Overview

This document covers the transformation of ICESat-2 photon data over a target reef to depth estimates that will be used to calibrate Sentinel-2 bathymetry estimates.  The relevant program files are located in the <i><b>/src</i></b> directory in Karan Sunil's "Coral-Reef-Bathymetry" distribution on GitHub: https://github.com/karans04/Coral-Reef-Bathymetry

## 1.) Import packages

To begin, make sure this notebook is located and launched from a working directory containing <i><b>Depth_profile.py</i></b> and the other python files in Karan's distribution. After the imports below, any function available in <i><b>Depth_profile.py</i></b> can be called from this notebook with the syntax: depth.function_name(parameters). The same goes for the other imported python files.

In [ ]:
import geopandas as gpd
import h5py
import importlib
import json
import numpy as np
import os
import pandas as pd
import pyproj as proj
import requests
import sys

from datetime import datetime
from sklearn.cluster import DBSCAN

import Coral_Reef as coral_reef
import Depth_profile as depth
import ICESAT_plots as is2_plot
import IS2_file as is2File
import Tide_API as tide
import Water_level as water_level

## 2.) Create directories and files needed for depth processing

In working directory, create a folder called <i><b>data</i></b> and create subfolders for each reef being analyzed.  Within each subfolder, provide the following folders/files:

<blockquote><b>A.)</b> a GeoJSON file containing the outline of the reef, named <i><b>reef_name.geojson</i></b>.  This file is obtained from http://geojson.io/#map=2/20.0/0.0.  Switch to "OSM model" in bottom left corner of window, mark the boundary of reef with the cursor, save points using the menu at upper left (save->GeoJSON), then rename map.geojson file and move to data directory.
<p>
<b>B.)</b> a folder named <i><b>H5</i></b> which contains ICESat-2 ATL03 data files for this reef.  These are HDF5 files that can be obtained from OpenAltimetry (http://www.openaltimetry.org) or from NASA EarthData search (https://search.earthdata.nasa.gov/search/granules?p=C1705401930-NSIDC_ECS)
<p>
<b>C.)</b> a file called <i><b>reef_name.text</i></b> containing metadata.  The format must be:<br>
<blockquote>Coordinates: <p>
[min_lat, max_lat, min_lon, max_lon] <br><br>H5 Files:<br>List of all h5 files in folder H5, one file per line.</blockquote></blockquote>

## 3.) Input metadata for reef processing

[run.py] <br>
Setting reef processing parameters can be done programatically via Karan's top-level processing script <i><b>run.py</i></b>, which updates <i><b>config/data-params.json</i></b>. Here these parameters are set via the command line.

In [ ]:
reef_name = 'nasau'
data_dir = '/Users/bonnieludka/Spaceship/IceSAT2/run_bathymetry/data/'
start_date = '20181101'
end_date = '20200601'
redownload_is2 = False  #False = use the data already downloaded into the directory
earthdata_login = 'bludka'
earthdata_password = 'Gnidlaps10!'
world_tide_API_key = 'fee8ff39-48eb-42a7-bcc5-3819fce3c1e4'

## 4.) Initialize Coral_Reef( ) data object

[run.py] <br>
Create a Coral_Reef object to hold metadata for each reef. Take a look in Coral_Reef.py for the methods and variables defined for the Coral_Reef class. Also note that the get_bounding_box( ) method queries <i><b>/data/reef_name/reef_name.txt</i></b> to grab the reef bounding box coordinates.

In [ ]:
reef = coral_reef.Coral_Reef(data_dir, reef_name)

## 5.) Get ICESat-2 data for reef

[run.py] <br>
This requires running <i><b>/src/IS2_file.main()</i></b>, which we are not going to do for this example.

## 6.) Initialize is2_file( ) data object

[run.py / Depth_profile.py / get_depths(reef)]<br>
Assuming for now that the ICESat-2 data have been downloaded and dropped into the <i><b>H5</i></b> directory, we will work with the <i><b>first</i></b> file for the example below.

In [ ]:
#Identify first ICESat-2 HDF5 file
h5_dir = os.path.join(reef.get_path(),'H5')
h5_fn = [f for f in os.listdir(h5_dir) if not f.startswith('.')]
h5_fn.sort()
h5_fn = h5_fn[0]
print(h5_dir)
print(h5_fn)

#Inititalize ICESat-2 file object
is2 = is2File.IS2_file(h5_dir, h5_fn, reef.bbox_coords)

## 7.) Estimate photon depths

[run_bathymetry.py / get_depths() / process_H5()]<br>
The depth.process_H5(reef) method will run the entire depth estimation workflow. Here it is line-by-line:

In [ ]:
#Get paths to all needed directories from reef object
icesat_fp, proc_fp, images_fp,data_plots_path = reef.get_file_drectories()
reef_name = reef.get_reef_name()

#Identify strong ICESat-2 beams
strong_beams = is2.get_strong_lasers()
print('These are the strong beams for this file: {}'.format(strong_beams))
is2_file_tag = is2.get_file_tag()

In [ ]:
importlib.reload(is2_plot)
importlib.reload(water_level)
importlib.reload(depth)

for laser in ['gt1r', 'gt1l', 'gt2r', 'gt2l', 'gt3r', 'gt3l']:
  
    #Output directory for csv file containing raw photon data
    photon_fn = '{reef_name}_photons_{h5_fn}_{laser}.csv'.format(reef_name=reef_name, h5_fn=is2_file_tag, laser=laser)
    photons_path = os.path.join(icesat_fp, photon_fn)
    print("\n" + photons_path)
    
    
    #Load raw photon data if it already exists, else extract it from h5 file
    if not os.path.exists(photons_path):

        #From process_h5(): 
        #df = depth.convert_h5_to_csv(is2, laser, photons_path)
        
        #Create dataframe with photon data
        photon_data = is2.get_photon_data(laser)
        
        h5 = h5py.File(is2.h5_file,'r')
        ##returns list of photon height, lat,lon and photon confidence
        height = h5.get(laser + '/heights/h_ph')
        lat = h5.get(laser + '/heights/lat_ph')
        lon = h5.get(laser + '/heights/lon_ph')
        conf = h5.get(laser + '/heights/signal_conf_ph')
        #print(conf)
        #print(is2.h5_file)
        #print(h5)
        confidence = np.array(conf)
        #print(confidence)
        #print( laser + '/heights/signal_conf_ph')
        #print(height)
        conf_land = confidence[:,0]
        conf_ocean = confidence[:,1]
        onf_inlandwater = confidence[:,3] 
        
        
        df_laser = depth.create_photon_df(photon_data)
        print(df_laser)

        #Clip dataframe to locations within the reef bounding box
        coords = is2.get_bbox_coordinates()
        min_longitude,min_latitude,max_longitude,max_latitude = coords
        df = df_laser.loc[(df_laser.Longitude > min_longitude) & (df_laser.Longitude < max_longitude) &\
            (df_laser.Latitude > min_latitude) & (df_laser.Latitude < max_latitude)]

        #If there are photons in the new clipped dataframe
        if len(df) != 0:    
            
            #Unpack the confidence array to individual columns (now done in get_photon_data
            #df = depth.individual_confidence(df)
            f = water_level.get_water_level(df)
            #is2_plot.plot_is2_depths(df, is2, laser, f, [0, 55], 'Raw Photon Elevs')
         
            #Adjust elevations to reference local sea level (currently this step ALSO subsets on confidence==4)
            #Unlike Karan's code, df now contains 'sea_level', which is what has been removed from 'Heights'
            df,f = water_level.normalise_sea_level(df)
            is2.set_sea_level_function(f,laser)
            if len(df) == 0:
                print('No photons')
                continue

            #Adjust depth for speed of light in water
            df = water_level.adjust_for_speed_of_light_in_water(df)
            #is2_plot.plot_is2_depths(df, is2, laser, 'False', [-25, 5], 'Adjusted Photon Elevs')

            #Write a dataframe containing just the photon data in bounding box that meet the confidence criterion
            print('Writing photons for clipped dataframe to csv file for {}'.format(laser))
            df.to_csv(photons_path)
        
        else:
            print('There are no photons in the clipped dataframe')
            
    else:
        print('Reading photons from prior csv file for {}'.format(laser))
        df = pd.read_csv(photons_path)
        is2.metadata = is2.load_json()
        
    #if no photons over reef for this laser, move onto next laser    
    if len(df) == 0:
        continue

    #Calculate predicted depths
    print('Number of ICESAT-2 Photons in {laser} is {reef_length}'.format(laser=laser, reef_length=str(len(df))))
    depths_fn = '{reef_name}_{h5_fn}_{laser}.csv'.format(reef_name=reef_name,h5_fn=is2_file_tag,laser=laser)
    processed_output_path = os.path.join(proc_fp,depths_fn)
 
    if laser in strong_lasers:
        eps_val = 1
        min_samp = 6
    else:
        eps_val = 1
        min_samp = 5
    
    bathy = depth.apply_DBSCAN(df, processed_output_path,is2, laser, eps_val, min_samp)
    is2_plot.plot_is2_depths_bathy(df, is2, laser, bathy, 'False', [-25, 5], 'Corrected Bathymetric Profile')

## Testing: Get tides

In [ ]:
epoch = datetime.utcfromtimestamp(0)
dt = is2.get_date()
ut = (dt - epoch).total_seconds()
print(epoch)
print(dt)
print(dt - epoch)
print(ut)

bbox = is2.bbox_coordinates
print(bbox)

In [ ]:
#tide_level = tide.get_tide(is2.bbox_coordinates, is2.get_date())
min_longitude,min_latitude,max_longitude,max_latitude = is2.bbox_coordinates
lat = str((min_latitude+max_latitude)/2)
lon = str((min_longitude+max_longitude)/2)
stadist = 100
base_url = 'https://www.worldtides.info/api/v2?heights'

tide_datum = 'MTL'
query_string = '&lat={lat}&lon={lon}&start={ut}&datum={datum}&key={api_key}&station_distance={stadist}'\
    .format(lat = lat,lon = lon,ut = ut,datum = tide_datum,api_key = tide.get_API_key(),stadist=stadist)
print(query_string)

In [ ]:
url = base_url + query_string
#hitting API and storing contents in json format
r = requests.get(url)
tide = json.loads(r.text)

In [ ]:
print(tide['heights'][0:1])
print(tide['responseLat'])
print(tide['responseLon'])
print(tide['atlas'])
print(tide['responseLat'])
print(tide['responseLon'])
print(tide['responseDatum'])